In [ ]:
import gspread
import pandas as pd
from datetime import datetime

In [ ]:
sa = gspread.service_account(filename="../credentials/jp-macdonnel-b7aa48547774.json")
sh = sa.open("green-lights")

master = sh.worksheet('master')
df_master = pd.DataFrame(master.get_all_records())

uone_export = open("../uone-exports/20230725.csv")
df_uone = pd.read_csv(uone_export)

In [ ]:
master_list = []

now = datetime.now().strftime("%B %d, %Y ~ %H:%M:%S")

for (
    i,
    customer_name,
    first_name,
    last_name,
    job_title,
    work_phone,
    email,
    address,
    city,
    state,
    zip,
    status,
    updated,
    protected_until,
    gl,
    gl_updated_at,
    gl_backfilled_at,
    lead_id,
    sales_stage,
) in df_master.itertuples():
    print(f"{i+1}. {customer_name}")
    if not gl:
        hit = df_uone.loc[df_uone["Customer Name"] == customer_name]

        if len(hit):
            ups_status = hit["UPS Status"].iloc[0]
            if type(ups_status) != float:
                master_list.append(
                    [
                        ups_status,
                        hit["Updated"].iloc[0],
                        now,
                        hit["Lead UONE Number"].iloc[0],
                        hit["Sales Stage"].iloc[0],
                    ]
                )
            else:
                master_list.append(
                    [
                        "",
                        "",
                        now,
                        hit["Lead UONE Number"].iloc[0],
                        hit["Sales Stage"].iloc[0],
                    ]
                )

        else:
            master_list.append(
                [
                    "",
                    "",
                    now,
                    "",
                    "",
                ]
            )
    else:
        master_list.append(
            [
                gl,
                gl_updated_at,
                gl_backfilled_at,
                lead_id,
                sales_stage,
            ]
        )


In [ ]:
display(master_list[-5:])

In [ ]:
master.update(f"N2:R{len(df_master) + 1}", master_list)